In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "unsloth/Llama-3.2-3B-bnb-4bit"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content) # no opening <think> tag
print("content:", content)


PackageNotFoundError: No package metadata was found for bitsandbytes

In [ ]:
import requests

def ollama_chat(model: str, messages: list):
    """
    Ping an Ollama model for chat completions.

    Args:
        model (str): Model name (e.g., 'llama3').
        messages (list): List of dicts with 'role' ('user' or 'assistant') and 'content'.

    Returns:
        str: Model's response.
    """
    url = "http://localhost:11434/api/chat"
    payload = {
        "model": model,
        "messages": messages
    }
    response = requests.post(url, json=payload)
    response.raise_for_status()
    return response.json()["message"]["content"]

# Example usage:
user_message = "Tell me about large language models."
assistant_message = "Large language models are AI systems trained on vast amounts of text data."

messages = [
    {"role": "user", "content": user_message},
    {"role": "assistant", "content": assistant_message}
]

reply = ollama_chat(model_name, messages)
print(reply)